In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [2]:
import os
print(os.getcwd())


/home/jovyan/DataDynomos/Code


In [3]:
final_df = pd.read_csv('/home/jovyan/DataDynomos/Datasets/final_df.csv')


In [4]:
final_df.sample()

,Unnamed: 0,img_path,DO_label,turbidity_label,label_code,DO_label_code
67,67,/content/resized_folder/resized_DSC_8941.jpg,Moderate,Very high turbidity,4,2


In [7]:
!unzip /home/jovyan/DataDynomos/Datasets/resized_images_512.zip -d /home/jovyan/DataDynomos/Datasets/resized_folder

In [10]:
from tensorflow.keras.applications import InceptionV3

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(512, 512, 3))


In [11]:
import os
import tensorflow as tf
import pandas as pd

# Assuming your DataFrame has 'img_path' and 'DO_label_code' columns
image_paths = final_df['img_path'].values
labels = final_df['DO_label_code'].values

# Create a list to store the full paths
full_image_paths = []

for img_path in image_paths:
    base_path = os.path.basename(img_path)
    full_image_path = os.path.join('/home/jovyan/DataDynomos/Datasets/resized_folder', base_path)
    full_image_paths.append(full_image_path)

# Now you can use full_image_paths and labels as needed


In [12]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)  # Adjust if images are PNG
    image = tf.image.resize(image, [512, 512])
    image = image / 255.0  # Normalize to [0, 1]
    return image

def load_image_and_label(path, label):
    return load_and_preprocess_image(path), label

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((full_image_paths, labels))
dataset = dataset.map(load_image_and_label, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=1000).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [14]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [15]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)  # Adjust the number of classes

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.002), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/10


2024-10-19 10:41:38.121239: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 536870912 exceeds 10% of free system memory.
2024-10-19 10:41:39.669773: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 536870912 exceeds 10% of free system memory.
2024-10-19 10:41:40.158016: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 545292288 exceeds 10% of free system memory.
2024-10-19 10:41:40.635943: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 536870912 exceeds 10% of free system memory.
2024-10-19 10:41:41.137338: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 536870912 exceeds 10% of free system memory.


12/12 [==============================] - ETA: 0s - loss: 3.8276 - accuracy: 0.2839 